<img src="monai.png" style="width: 700px;"/>

Welcome to the MONAI bootcamp! This tutorial introduces the basic concepts of the MONAI Deploy App SDK by showing how to develop a simple image processing application.

### Using Google Colab

This notebook has the pip command for installing MONAI and will be added to any subsequent notebook.

**Required Packages for Colab Execution**

Execute the following cell to install MONAI the first time a colab notebook is run:

In [ ]:
!python -c "import skimage" || pip install -qU "scikit-image"
!python -c "import monai" || pip install -qU "monai[nibabel]"
!python -c "import monai.deploy" || pip install -qU "monai-deploy-app-sdk"

**Note:** If you're having issues with imports, try to "Restart Kernel" for Jupyter from the "Kernel" dropdown menu.

**Enabling GPU Support**

To use GPU resources through Colab, change the runtime to GPU:

1. From the **"Runtime"** menu select **"Change Runtime Type"**
2. Choose **"GPU"** from the drop-down menu
3. Click **"SAVE"**

This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not). Running

**!nvidia-smi**

in a cell will verify this has worked and show you what kind of hardware you have access to.    

In [ ]:
!nvidia-smi

# Creating a Simple Image Processing App with MONAI Deploy App SDK

First, you will need to design the workflow of your application that defines Operators (tasks) and flows among them. Once the workflow is designed, you can use existing operators provided by MONAI Deploy or start implementing your customer operator classes. Next, you'll implement an Application class to construct a workflow graph with the operators.

![MONAI Deploy App SDK Workflow](images/sdk_workflow.png)

You can execute and debug your application locally in a Jupyter notebook or CLI.

### Contents
* [Setup](#setup)
* [Creating Operator Class](#create_op)
* [Creating Application Class](#create_app)
* [Executing App Locally](#execute_app_local)
* [Write Files to Disk](#write_files)
* [Run App using CLI](#run_cli)
* [Packing App as Docker Image](#package_app)
* [Executing Packaged App Locally](#execute_docker)
* [Conclusion](#conclusion)

## Creating Operators and connecting them in the Application class

We will implement an application that consists of three Operators:

- **SobelOperator**: Apply a Sobel edge detector.
    - **Input**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **MedianOperator**: Apply a Median filter for noise reduction.
    - **Input**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **GaussianOperator**: Apply a Gaussian filter for smoothening.
    - **Input**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    
The workflow of the application would look like this.

![Application Workflow](images/app_workflow.png)

<a id='setup'></a>
## 0 Setup

### 0.1 Setup environment

We'll set up a variable to point to the 01_files folder, where we're storing data throughout this notebook.

In [ ]:
NOTEBOOK_ROOT="data/01_files/"

**Note:** If you're having issues with imports, try to "Restart Kernel" for Jupyter from the "Kernel" dropdown menu.

#### Download Data required for all of the MONAI Deploy Notebooks
We're going to download sample images and python files required for running the MONAI Deploy Jupyter Notebooks.

In [ ]:
!wget https://github.com/zephyrie/monai-bootcamp/releases/download/monai-deploy-data-v0.1/deploy_data.zip
!unzip deploy_data.zip

### 0.2 Download test input

We will use a test input from the following.

> Case courtesy of Dr Bruno Di Muzio, <a href="https://radiopaedia.org/?lang=us">Radiopaedia.org</a>. From the case <a href="https://radiopaedia.org/cases/41113?lang=us">rID: 41113</a>

In [ ]:
from skimage import io
# import wget

test_input_path = NOTEBOOK_ROOT+"input/normal-brain-mri-4.png"
# wget.download("https://user-images.githubusercontent.com/1928522/133383228-2357d62d-316c-46ad-af8a-359b56f25c87.png", test_input_path)

print(f"Test input file path: {test_input_path}")

test_image = io.imread(test_input_path)
io.imshow(test_image)

### 0.3 Setup imports

Let's import necessary classes/decorators to define Application and Operator.

In [ ]:
import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)

<a id='create_op'></a>
## 1. Creating Operator classes

Each Operator class inherits [Operator](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html) class and input/output properties are specified by using [@input](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.input.html)/[@output](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.output.html) decorators.

Note that the first operator(SobelOperator)'s input and the last operator(GaussianOperator)'s output are [DataPath](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html) type with [IOType.DISK](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.IOType.html). Those paths are mapped into input and output paths given by the user during the execution.

Business logic would be implemented in the <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html#monai.deploy.core.Operator.compute">compute()</a> method.

### 1.1 SobelOperator

SobelOperator is the first operator (A root operator in the workflow graph). <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.InputContext.html#monai.deploy.core.InputContext.get">op_input.get(label)</a> (since only one input is defined in this operator, we don't need to specify an input label) would return an object of [DataPath](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html) and the input file/folder path would be available by accessing the `path` property (`op_input.get().path`).

Once an image data (as a Numpy array) is loaded and processed, [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) object is created from the image data and set to the output (<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.OutputContext.html#monai.deploy.core.OutputContext.set">op_output.set(value, label)</a>).

In [ ]:
@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
# If `pip_packages` is specified, the definition will be aggregated with the package dependency list of other
# operators and the application in packaging time.
# @md.env(pip_packages=["scikit-image >= 0.17.2"])
class SobelOperator(Operator):
    """This Operator implements a Sobel edge detector.

    It has a single input and single output.
    """

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage import filters, io

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        data_in = io.imread(input_path)[:, :, :3]  # discard alpha channel if exists
        data_out = filters.sobel(data_in)

        op_output.set(Image(data_out))

### 1.2 MedianOperator

MedianOperator is a middle operator that accepts data from SobelOperator and passes the processed image data to GaussianOperator.

Its input and output data types are [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) and the Numpy array data is available through `asnumpy()` method (`op_input.get().asnumpy()`).

Again, once an image data (as a Numpy array) is loaded and processed, [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) object is created and set to the output (<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.OutputContext.html#monai.deploy.core.OutputContext.set">op_output.set(value, label)</a>).

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("image", Image, IOType.IN_MEMORY)
# If `pip_packages` is specified, the definition will be aggregated with the package dependency list of other
# operators and the application in packaging time.
# @md.env(pip_packages=["scikit-image >= 0.17.2"])
class MedianOperator(Operator):
    """This Operator implements a noise reduction.

    The algorithm is based on the median operator.
    It ingests a single input and provides a single output.
    """

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage.filters import median

        data_in = op_input.get().asnumpy()
        data_out = median(data_in)
        op_output.set(Image(data_out))

### 1.3 GaussianOperator

GaussianOperator is the last operator (A leaf operator in the workflow graph) and the output path of this operator is mapped to the user-provided output folder so we cannot set a path to `op_output` variable (e.g., `op_output.set(Image(data_out))`).

Instead, we can get the output path through `op_output.get().path` and save the processed image data into a file.

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("image", DataPath, IOType.DISK)
# If `pip_packages` is specified, the definition will be aggregated with the package dependency list of other
# operators and the application in packaging time.
# @md.env(pip_packages=["scikit-image >= 0.17.2"])
class GaussianOperator(Operator):
    """This Operator implements a smoothening based on Gaussian.

    It ingests a single input and provides a single output.
    """

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage.filters import gaussian
        from skimage.io import imsave

        data_in = op_input.get().asnumpy()
        data_out = gaussian(data_in, sigma=0.2)

        output_folder = op_output.get().path
        output_path = output_folder / "final_output.png"
        imsave(output_path, data_out)

<a id='create_app'></a>
## 2. Creating Application class

Our application class would look like below.

It defines `App` class, inheriting [Application](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html) class.

The requirements (resource and package dependency) for the App can be specified by using [@resource](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.resource.html) and [@env](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.env.html) decorators.

The @resource and @env decorators are also helpful from a Deployment IT / Infrastructure perspective in that they help with reproducibility and infrastructure requirements directly in the code.

In [ ]:
@md.resource(cpu=1)
# pip_packages can be a string that is a path(str) to requirements.txt file or a list of packages.
@md.env(pip_packages=["scikit-image >= 0.17.2"])
class App(Application):
    """This is a very basic application.

    This showcases the MONAI Deploy application framework.
    """

    # App's name. <class name>('App') if not specified.
    name = "simple_imaging_app"
    # App's description. <class docstring> if not specified.
    description = "This is a very simple application."
    # App's version. <git version tag> or '0.0.0' if not specified.
    version = "0.1.0"

    def compose(self):
        """This application has three operators.

        Each operator has a single input and a single output port.
        Each operator performs some kind of image processing function.
        """
        sobel_op = SobelOperator()
        median_op = MedianOperator()
        gaussian_op = GaussianOperator()

        self.add_flow(sobel_op, median_op)
        # self.add_flow(sobel_op, median_op, {"image": "image"})
        # self.add_flow(sobel_op, median_op, {"image": {"image"}})

        self.add_flow(median_op, gaussian_op)

In `compose()` method, objects of `SobelOperator`, `MedianOperator`, and `GaussianOperator` classes are created
and connected through <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a>.

> add_flow(upstream_op, downstream_op, io_map=None)

`io_map` is a dictionary of mapping from the source operator's label to the destination operator's label(s) and its type is `Dict[str, str|Set[str]]`. 

We can skip specifying `io_map` if both the number of `upstream_op`'s outputs and the number of `downstream_op`'s inputs are one so `self.add_flow(sobel_op, median_op)` is same with `self.add_flow(sobel_op, median_op, {"image": "image"})` or `self.add_flow(sobel_op, median_op, {"image": {"image"}})`.


<a id='execute_app_local'></a>
## 3. Executing app locally

Once an Application class (App) is defined, you can instantiate the Application and execute with the Application.run() method.

Utilizing Jupyter Notebooks with MONAI Deploy App SDK gives developers an easy way to get started building AI applications.  You have the full suite of Jupyter tools for debugging and fast iteration and then can quickly transition to creating a full-fledged AI Application docker container using the same tools.

In [ ]:
app = App()
app.run(input=test_input_path, output=NOTEBOOK_ROOT+"/output")

### 3.1 Show results 

We'll then use Matplotlib to show the output result.

In [ ]:
!ls -la $NOTEBOOK_ROOT/output
output_image = io.imread(NOTEBOOK_ROOT+"output/final_output.png")
# io.imshow(output_image)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 15 ,15
fig, ax = plt.subplots(1,2)

ax[0].imshow(test_image);
ax[1].imshow(output_image);

<a id='write_file'></a>
## 4. Write files to Disk

Once the application is verified inside the Jupyter notebook, we can write the above Python code into Python files in an application folder.

The application folder structure would look like below:

```bash
simple_imaging_app
├── __main__.py
├── app.py
├── gaussian_operator.py
├── median_operator.py
└── sobel_operator.py
```

---
 > **Note**: We can create a single application Python file (such as `simple_imaging_app.py`) that includes the files' content instead of creating multiple files.
You will see such example in <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/notebooks/tutorials/02_mednist_app.html#executing-app-locally">MedNist Classifier Tutorial</a>.
---

Usually, you would have an IDE and directly edit files on disk. However, we have been editing code in the jupyter cells in this lab. We will now write code in the cells to files using the Jupyter magic command `%%writefile`. You will be directly editing Python code in the Jupyter editor in future labs.

We'll start by creating an application folder called `simple_imaging_app` and then writing each operator, app and main Python files to that directory.

In [ ]:
# Create an application folder
!mkdir -p {NOTEBOOK_ROOT}/simple_imaging_app

### 4.1 Write sobel_operator.py to file 

In [ ]:
%%writefile {NOTEBOOK_ROOT}/simple_imaging_app/sobel_operator.py
import monai.deploy.core as md
from monai.deploy.core import (
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)


@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
class SobelOperator(Operator):
    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage import filters, io

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        data_in = io.imread(input_path)[:, :, :3]  # discard alpha channel if exists
        data_out = filters.sobel(data_in)

        op_output.set(Image(data_out))

### 4.2 Write median_operator.py to file 

In [ ]:
%%writefile {NOTEBOOK_ROOT}/simple_imaging_app/median_operator.py
import monai.deploy.core as md
from monai.deploy.core import ExecutionContext, Image, InputContext, IOType, Operator, OutputContext, output


@md.input("image", Image, IOType.IN_MEMORY)
@md.output("image", Image, IOType.IN_MEMORY)
class MedianOperator(Operator):
    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage.filters import median

        data_in = op_input.get().asnumpy()
        data_out = median(data_in)
        op_output.set(Image(data_out))

### 4.3 Write gaussian_operator.py to file 

In [ ]:
%%writefile {NOTEBOOK_ROOT}/simple_imaging_app/gaussian_operator.py
import monai.deploy.core as md
from monai.deploy.core import (
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)


@md.input("image", Image, IOType.IN_MEMORY)
@md.output("image", DataPath, IOType.DISK)
class GaussianOperator(Operator):
    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage.filters import gaussian
        from skimage.io import imsave

        data_in = op_input.get().asnumpy()
        data_out = gaussian(data_in, sigma=0.2)

        output_folder = op_output.get().path
        output_path = output_folder / "final_output.png"
        imsave(output_path, data_out)

### 4.4 Write app.py to file 

In [ ]:
%%writefile {NOTEBOOK_ROOT}/simple_imaging_app/app.py
import monai.deploy.core as md
from gaussian_operator import GaussianOperator
from median_operator import MedianOperator
from sobel_operator import SobelOperator

from monai.deploy.core import Application


@md.resource(cpu=1)
@md.env(pip_packages=["scikit-image >= 0.17.2"])
class App(Application):
    def compose(self):
        sobel_op = SobelOperator()
        median_op = MedianOperator()
        gaussian_op = GaussianOperator()

        self.add_flow(sobel_op, median_op)
        self.add_flow(median_op, gaussian_op)

# Run the application when this file is executed.
if __name__ == "__main__":
    App(do_run=True)

```python
if __name__ == "__main__":
    App(do_run=True)
```

The above lines are needed to execute the application code by using `python` interpreter.

### 4.5 Write  \_\_main\_\_.py to file 

\_\_main\_\_.py is needed for <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/packaging_app.html">MONAI Application Packager</a> to detect the main application code (`app.py`) when the application is executed with the application folder path (e.g., `python simple_imaging_app`).

In [ ]:
%%writefile {NOTEBOOK_ROOT}/simple_imaging_app/__main__.py
from app import App

if __name__ == "__main__":
    App(do_run=True)

In [ ]:
# check files exist
!ls -la {NOTEBOOK_ROOT}simple_imaging_app

<a id='run_cli'></a>
## 5. Run App on CLI
You can run the App in one of two ways:
- Using the python command
- Using the 'monai-deploy' command 

---
 > **Note**: We are executing python code which makes developing and debugging simple.
---

In [ ]:
!python {NOTEBOOK_ROOT}/simple_imaging_app -i {test_input_path} -o {NOTEBOOK_ROOT}"/output"

The above command and below command both run the application, but one uses python directly, and the other uses the provided MONAI Deploy CLI.

In [ ]:
!monai-deploy exec {NOTEBOOK_ROOT}/simple_imaging_app -i {test_input_path} -o {NOTEBOOK_ROOT}"/output"

We'll then use Matplotlib again to show the output result.

In [ ]:
output_image = io.imread(NOTEBOOK_ROOT+"/output/final_output.png")
io.imshow(output_image)

<a id='package_app'></a>
## 6. Package app (creating MAP Docker image)

### 6.1 Docker intro 

Why [Docker](https://docs.docker.com/get-started/overview/)? It's lightweight and reproducible. While VMs encapsulate the entire OS and any applications, containers encapsulate individual applications and their dependencies for portable deployment, but share the same host OS between containers.

![Docker versus VM](images/docker.png)

Examples:
- This DLI is using docker
- NVIDIA releases docker for each framework every month

Let's package the app with <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/packaging_app.html">MONAI Application Packager</a>.

The MONAI Application Packager (Packager) is a utility for building an application developed with the MONAI Deploy App SDK into a structured MONAI Application Package (MAP).

The MAP produced by the Packager is a deployable and reusable docker image that can be launched locally or remotely.

### 6.2 Basic Usage of MONAI Application Packager

Below we'll use the monai-deploy CLI to package the application by using the package command.  We'll pass the `simple_imaging_app` folder path and use the `simple_app` tag.

In [ ]:
#!monai-deploy package {NOTEBOOK_ROOT}/simple_imaging_app --tag simple_app:latest -l DEBUG

---
 > **Note**: Building a MONAI Application Package (Docker image) can take time. Use `-l DEBUG` option if you want to see the progress.
---

After creating the docker image, we can verify it by using the `docker image ls` command and grepping for `simple_app`.

In [ ]:
#!docker image ls | grep simple_app

<a id='execute_docker'></a>
## 7. Executing packaged app locally

The packaged app can be run locally using the [MONAI Application Runner](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/executing_packaged_app_locally.html).

---
 > **Note**: Following command will run on the host machine using docker. 
Therefore: 
> - any variables defined in the notebook as `NOTEBOOK_ROOT` won't resolve
> - mapping volumes should be the real host path, that is why we use the docker variable LAB_PATH
---


In [ ]:
# Launch the app
#!monai-deploy run simple_app:latest "${NOTEBOOK_ROOT}/input" "${NOTEBOOK_ROOT}/dockerOutput"

#### 7.1 Show Results from docker package

We'll then use Matplotlib again to show the output result.

In [ ]:
#output_image = io.imread(NOTEBOOK_ROOT+"/dockerOutput/final_output.png")
#io.imshow(output_image)

<a id='conclusion'></a>
## Conclusion

In this notebook, we have walked through the process to create a simple application using MONAI Deploy App SDK.  You've run the application in Jupyter, locally using Python and the MONAI Deploy CLI, and finally you packaged the application using docker and executed the newly created container image.

### What's Next
The following notebook we'll walk through a more realistic classification task using the MedNIST dataset.